<a href="https://colab.research.google.com/github/YasJanam/NLP_MODELS_1/blob/main/Labratory_10/Labratory_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lightning evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.2/825.2 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.2/829.2 kB 57.1 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=bf491219fafb484e116f353ea5fdbe8b85315a3717e3a4b7b0cb91f9ab24dc87
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer
import lightning.pytorch as pl
from lightning.pytorch.loggers import CSVLogger
import evaluate

### **BART-Model**

In [3]:
class FFN(nn.Module):
  def __init__(self,d_model,d_ff,dropout=0.1):
    super().__init__()
    self.lin1 = nn.Linear(d_model,d_ff)
    self.lin2 = nn.Linear(d_ff,d_model)
    self.dropout = nn.Dropout(dropout)
  def forward(self,x):
    return self.lin2(self.dropout(F.gelu(self.lin1(x))))

class SelfAttn(nn.Module):
  def __init__(self,d_model,n_heads,dropout=0.1):
    super().__init__()
    self.attn = nn.MultiheadAttention(d_model,n_heads,dropout=dropout,batch_first=True)
    self.ln = nn.LayerNorm(d_model)
  def forward(self,x,attn_mask=None,key_padding_mask=None):
    h,_ = self.attn(x,x,x,attn_mask=attn_mask,key_padding_mask=key_padding_mask,need_weights=False)
    return self.ln(x+h)

class CrossAttn(nn.Module):
  def __init__(self,d_model,n_heads,dropout=0.1):
    super().__init__()
    self.attn = nn.MultiheadAttention(d_model,n_heads,dropout=dropout,batch_first=True)
    self.ln = nn.LayerNorm(d_model)
  def forward(self,x,mem,attn_mask=None,key_padding_mask=None,mem_padding_mask=None):
    h,_ = self.attn(x,mem,mem,attn_mask=attn_mask,key_padding_mask=key_padding_mask)
    return self.ln(x+h)

class EncoderLayer(nn.Module):
  def __init__(self,d_model,n_heads,d_ff,dropout=0.1):
    super().__init__()
    self.self_attn = SelfAttn(d_model,n_heads,dropout)
    self.ffn = FFN(d_model,d_ff,dropout)
    self.ln = nn.LayerNorm(d_model)
  def forward(self,x,key_padding_mask=None):
    x = self.self_attn(x,key_padding_mask=key_padding_mask)
    return self.ln(x+self.ffn(x))

class DecoderLayer(nn.Module):
  def __init__(self,d_model,n_heads,d_ff,dropout=0.1):
    super().__init__()
    self.self_attn = SelfAttn(d_model,n_heads,dropout)
    self.cross_attn = CrossAttn(d_model,n_heads,dropout)
    self.ffn = FFN(d_model,d_ff,dropout)
    self.ln = nn.LayerNorm(d_model)
  def forward(self,x,mem,tgt_key_padding_mask=None,mem_key_padding_mask=None,causal_mask=None):
    x = self.self_attn(x,attn_mask=causal_mask,key_padding_mask=tgt_key_padding_mask)
    x = self.cross_attn(x,mem,mem_padding_mask=mem_key_padding_mask)
    return self.ln(x+self.ffn(x))

In [5]:
class MiniBART(nn.Module):
  def __init__(self,vocab_size,d_model=64,n_heads=2,d_ff=512,num_enc=2,num_dec=2,max_len=128):
    super().__init__()
    self.tok_emb = nn.Embedding(vocab_size,d_model)
    self.pos_emb = nn.Embedding(max_len,d_model)
    self.enc_layers = nn.ModuleList([EncoderLayer(d_model,n_heads,d_ff) for _ in range(num_enc)])
    self.dec_layers = nn.ModuleList([DecoderLayer(d_model,n_heads,d_ff) for _ in range(num_dec)])
    self.lm_head = nn.Linear(d_model,vocab_size,bias=False)

  def forward(self,src_ids,tgt_ids):

    def add_pos(x):
      b,L = x.shape
      pos = torch.arange(L,device=x.device).unsqueeze(0).expand(b,L)
      return self.tok_emb(x) + self.pos_emb(pos)

    src = add_pos(src_ids)
    tgt = add_pos(tgt_ids)

    # encoder
    mem = src
    for layer in self.enc_layers:
      mem = layer(mem)

    # causal mask for decoder self-attn
    L = tgt.size(1)
    causal = torch.triu(torch.ones(L,L,device=tgt.device)*float("-inf"),diagonal=1)

    out = tgt
    for layer in self.dec_layers:
      out = layer(out,mem,causal_mask=causal)

    logits = self.lm_head(out)
    return logits

### **Dataset**

In [6]:
dataset = load_dataset("cnn_dailymail","3.0.0")

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [7]:
MAX_INPUT = 128
MAX_TARGET = 64

def prepprocess(batch):
  inputs = tokenizer(
      batch["article"],max_length=MAX_INPUT,padding="max_length",truncation=True
  )
  targets = tokenizer(
      batch["highlights"],max_length=MAX_TARGET,padding="max_length",truncation=True
  )
  inputs["labels"] = targets["input_ids"]
  return inputs

برای این مدل 3000 داده آموزشی خیلی کم است.برای نتیجه بهتر داده آموزشی را بهتر کنید

In [13]:
train_dataset = dataset["train"].select(range(20000))
val_dataset = dataset["validation"].select(range(4000))

tokenized_train_data = train_dataset.map(prepprocess,batched=True,remove_columns=train_dataset.column_names)
tokenized_val_data = val_dataset.map(prepprocess,batched=True,remove_columns=val_dataset.column_names)

train_data = tokenized_train_data.with_format("torch")
val_data = tokenized_val_data.with_format("torch")

train_loader = DataLoader(train_data,batch_size=4,shuffle=True)
val_loader = DataLoader(val_data,batch_size=4,shuffle=False)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

### **Lightning Module**

In [14]:
rouge = evaluate.load("rouge")

class LitMiniBART(pl.LightningModule):
  def __init__(self,vocab_size,d_model=256,n_heads=4,d_ff=1024,num_enc=3,num_dec=3):
    super().__init__()
    self.model = MiniBART(vocab_size,d_model,n_heads,d_ff,num_enc,num_dec)
    self.loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
    self.val_preds = []
    self.val_labels = []

  def forward(self,src_ids,tgt_ids):
    return self.model(src_ids,tgt_ids)

  def training_step(self,batch,batch_idx):
    logits = self(batch["input_ids"],batch["labels"])
    shift_logits = logits[:,:-1].contiguous()
    shift_labels = batch["labels"][:,1:].contiguous()
    loss = self.loss_fn(shift_logits.view(-1,shift_logits.size(-1)),shift_labels.view(-1))
    self.log("train_loss",loss,prog_bar=True)
    return loss

  def validation_step(self,batch,batch_idx):
    logits = self(batch["input_ids"],batch["labels"])
    shift_logits = logits[:,:-1].contiguous()
    shift_labels = batch["labels"][:,1:].contiguous()
    loss = self.loss_fn(shift_logits.view(-1,shift_logits.size(-1)),shift_labels.view(-1))
    self.log("val_loss",loss,prog_bar=True)

    generated_ids = torch.argmax(logits,dim=-1)
    preds = tokenizer.batch_decode(generated_ids,skip_special_tokens=True)
    labels = tokenizer.batch_decode(batch["labels"],skip_special_tokens=True)
    self.val_preds.extend(preds)
    self.val_labels.extend(labels)

  def on_validation_epoch_end(self) :
    if len(self.val_preds) > 0:
      results = rouge.compute(predictions=self.val_preds,references=self.val_labels,use_stemmer=True)
      self.log_dict({f"val_{k}":v for k,v in results.items()},prog_bar=True)
      self.val_preds = []
      self.val_labels = []

  def configure_optimizers(self) :
    return torch.optim.AdamW(self.parameters(),lr=5e-4)

### **Train_1**

In [15]:
logger = CSVLogger("logs",name="MiniBART")

model = LitMiniBART(vocab_size=tokenizer.vocab_size)
trainer = pl.Trainer(
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    max_epochs=1,
    precision=16 if torch.cuda.is_available() else 32,
    logger=logger,
    log_every_n_steps=50
)

INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [16]:
trainer.fit(model,train_loader,val_loader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | model   | MiniBART         | 31.3 M | train
1 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
31.3 M    Trainable params
0         Non-trainable params
31.3 M    Total params
125.192   Total estimated model params size (MB)
79        Modules in train mode
0         Modules in eval mode
INFO:lightning.pytorch.callbacks.model_summary:
  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | model   | MiniBART         | 31.3 M | train
1 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
31.3 M    Trainable params
0         Non-trainable params
31.3 M    Total params
125.192   Total estimated model para

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


### **Train_2**

In [18]:
train_dataset_2 = dataset["train"].select(range(20000, 50000))
val_dataset_2 = dataset["validation"].select(range(4000, 9000))

tokenized_train_data_2 = train_dataset_2.map(prepprocess,batched=True,remove_columns=train_dataset_2.column_names)
tokenized_val_data_2 = val_dataset_2.map(prepprocess,batched=True,remove_columns=val_dataset_2.column_names)

train_data_2 = tokenized_train_data_2.with_format("torch")
val_data_2 = tokenized_val_data_2.with_format("torch")

train_loader_2 = DataLoader(train_data_2,batch_size=4,shuffle=True)
val_loader_2 = DataLoader(val_data_2,batch_size=4,shuffle=False)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [20]:
trainer_2 = pl.Trainer(
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    max_epochs=2,
    precision=16 if torch.cuda.is_available() else 32,
    logger=logger,
    log_every_n_steps=50
)

/usr/local/lib/python3.12/dist-packages/lightning/fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.util

In [21]:
trainer_2.fit(model,train_loader_2,val_loader_2)

/usr/local/lib/python3.12/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:701: Checkpoint directory logs/MiniBART/version_1/checkpoints exists and is not empty.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.12/dist-packages/lightning/pytorch/utilities/model_summary/model_summary.py:231: Precision 16-mixed is not supported by the model summary.  Estimated model size in MB will not be accurate. Using 32 bits instead.
INFO: 
  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | model   | MiniBART         | 31.3 M | train
1 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
31.3 M    Trainable params
0         Non-trainable params
31.3 M    Total params
125.192   Total estimated model params size (MB)
79        Modules in train mode
0         Modules in eval mode
INFO:li

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=2` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=2` reached.


### **Train_3**

In [22]:
train_dataset_3 = dataset["train"].select(range(50000, 70000))
val_dataset_3 = dataset["validation"].select(range(9000, 13000))

tokenized_train_data_3 = train_dataset_3.map(prepprocess,batched=True,remove_columns=train_dataset_2.column_names)
tokenized_val_data_3 = val_dataset_3.map(prepprocess,batched=True,remove_columns=val_dataset_2.column_names)

train_data_3 = tokenized_train_data_3.with_format("torch")
val_data_3 = tokenized_val_data_3.with_format("torch")

train_loader_3 = DataLoader(train_data_3,batch_size=4,shuffle=True)
val_loader_3 = DataLoader(val_data_3,batch_size=4,shuffle=False)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [23]:
trainer_3 = pl.Trainer(
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    max_epochs=5,
    precision=16 if torch.cuda.is_available() else 32,
    logger=logger,
    log_every_n_steps=50
)

/usr/local/lib/python3.12/dist-packages/lightning/fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.util

In [24]:
trainer_3.fit(model,train_loader_3,val_loader_3)

/usr/local/lib/python3.12/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:701: Checkpoint directory logs/MiniBART/version_1/checkpoints exists and is not empty.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.12/dist-packages/lightning/pytorch/utilities/model_summary/model_summary.py:231: Precision 16-mixed is not supported by the model summary.  Estimated model size in MB will not be accurate. Using 32 bits instead.
INFO: 
  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | model   | MiniBART         | 31.3 M | train
1 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
31.3 M    Trainable params
0         Non-trainable params
31.3 M    Total params
125.192   Total estimated model params size (MB)
79        Modules in train mode
0         Modules in eval mode
INFO:li

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=5` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


### **Train_4**

In [42]:
train_dataset_4 = dataset["train"].select(range(70000, 100000))
val_dataset_4 = dataset["validation"].select(range(10000, 13000))

tokenized_train_data_4 = train_dataset_4.map(prepprocess,batched=True,remove_columns=train_dataset_4.column_names)
tokenized_val_data_4 = val_dataset_4.map(prepprocess,batched=True,remove_columns=val_dataset_4.column_names)

train_data_4 = tokenized_train_data_4.with_format("torch")
val_data_4 = tokenized_val_data_4.with_format("torch")

train_loader_4 = DataLoader(train_data_4,batch_size=4,shuffle=True)
val_loader_4 = DataLoader(val_data_4,batch_size=4,shuffle=False)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [43]:
trainer_4 = pl.Trainer(
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    max_epochs=3,
    precision=16 if torch.cuda.is_available() else 32,
    logger=logger,
    log_every_n_steps=50
)

/usr/local/lib/python3.12/dist-packages/lightning/fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.util

In [44]:
trainer_4.fit(model,train_loader_4,val_loader_4)

/usr/local/lib/python3.12/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:701: Checkpoint directory logs/MiniBART/version_1/checkpoints exists and is not empty.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.12/dist-packages/lightning/pytorch/utilities/model_summary/model_summary.py:231: Precision 16-mixed is not supported by the model summary.  Estimated model size in MB will not be accurate. Using 32 bits instead.
INFO: 
  | Name    | Type             | Params | Mode
----------------------------------------------------
0 | model   | MiniBART         | 31.3 M | eval
1 | loss_fn | CrossEntropyLoss | 0      | eval
----------------------------------------------------
31.3 M    Trainable params
0         Non-trainable params
31.3 M    Total params
125.192   Total estimated model params size (MB)
0         Modules in train mode
79        Modules in eval mode
INFO:lightni

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=3` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


### **Test**

In [45]:
def greedy_decode(model,src_text,max_len=64):
  model.eval()
  src_ids = tokenizer(src_text,return_tensors="pt",truncation=True, padding="max_length",max_length=MAX_INPUT)["input_ids"].to(model.device)
  tgt_ids = torch.tensor([[tokenizer.bos_token_id]]).to(model.device)


  for _ in range(max_len):
    logits = model(src_ids,tgt_ids)
    next_token = logits[:,-1].argmax(-1).unsqueeze(0)
    tgt_ids = torch.cat([tgt_ids, next_token],dim=1)
    if next_token.item() == tokenizer.eos_token_id:
      break
  return tokenizer.decode(tgt_ids.squeeze(),skip_special_tokens=True)

In [46]:
print(greedy_decode(model, dataset["test"][9]["article"]))

The U.S. is the first U.S. to lead a new Ebola outbreak in the U.S.
The U.S. is the first U.S. in the U.S. in the U.S.
U.S. is the first U.S. in


نتیجه -> نامفهوم